# Qual aluno tem menor desvio padrão de notas?

In [57]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, inspect 

In [58]:
engine = create_engine('sqlite:///:memory:') # Criando banco SQL
engine 

Engine(sqlite:///:memory:)

In [59]:
tabela_notas = pd.read_excel('files/tabela_nota.xlsx') # Importando tabela de notas
tabela_notas.drop(tabela_notas.loc[tabela_notas['id_materia']==0].index, inplace=True) # Não é possivel saber qual materia ele fez
tabela_notas['id'] = range(tabela_notas.shape[0])
tabela_notas.set_index('id', inplace=True)
tabela_notas.index = tabela_notas.index + 1
tabela_notas.to_sql('notas', engine) 
inspector = inspect(engine) 
print(inspector.get_table_names())

['notas']


In [60]:
tabela_alunos = pd.read_excel('files/tabela_estudante.xlsx', index_col=False) # Importando tabela de alunos
tabela_alunos.set_index('Unnamed: 0', inplace=True)
tabela_alunos.index.rename('id', inplace=True)

## Desvio padrão de nota por aluno

In [61]:
# Função que seleciona o aluno para fazer o calculo do desvio padrao das notas
def seleciona_aluno(nome: str) -> int:
    aluno = tabela_alunos[tabela_alunos['nome'] == nome].index
    aluno = aluno[0]
    return aluno

# Funcao que busca a tabela de notas do aluno selecionado
def busca_tabela_notas_aluno(nome_aluno: str):
    query = f'select * from notas where id_estudante = {seleciona_aluno(nome_aluno)}'
    notas_aluno_selecionado = pd.read_sql(query, engine)
    return notas_aluno_selecionado

# Pega o nome do aluno selecionado
def traz_nome_aluno(id_aluno: int) -> str:
    nome_aluno = tabela_alunos['nome'][id_aluno]
    return nome_aluno

# Transforma df em lista 
def pega_lista_de_notas_aluno(notas_aluno_selecionado):
    notas = []
    notas.append(notas_aluno_selecionado['nota'].tolist())
    return notas_aluno

def calcula_desvio_padrao(nome_aluno: str) -> float:
    notas_aluno_selecionado = busca_tabela_notas_aluno(nome_aluno)
    desvio_padrao_notas = notas_aluno_selecionado['nota'].std()
    return desvio_padrao_notas

# Exportar no excel
def exporta_para_excel(itens: list, nome_coluna_itens: str, valores: list, nome_coluna_valores: str, caminho: str):
    dados = pd.DataFrame({nome_coluna_itens: itens, nome_coluna_valores: valores})
    dados.to_excel(caminho, index=False)

### Francisco Alves 

In [62]:
desvio_padrao_notas = []
qtd_alunos = tabela_alunos.shape[0] + 1

for i in range(1, qtd_alunos):
    desvio_padrao_notas.append(calcula_desvio_padrao(traz_nome_aluno(i)))

desvio_padrao_notas = pd.DataFrame(desvio_padrao_notas).fillna(0)
desvio_padrao_notas = list(desvio_padrao_notas[0])

In [63]:
estudantes = list(tabela_alunos['nome'])

In [64]:
exporta_para_excel(estudantes, 'Estudante', desvio_padrao_notas, 'Desvio Padrão de Notas', 'files/files_questions/question_6/desvio_padrao_notas_alunos.xlsx')